In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium import webdriver

In [2]:
class RottenCrawler:

    def __init__(self, df, headless):
        self.dataset = df
        driver_options = webdriver.ChromeOptions()
        if headless:
            driver_options.add_argument("--headless")
        self.driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=driver_options)

    @staticmethod
    def rottenize(title):a nao ser q o site do rotten 
        """
        Esta funcao sustitui os carateres por vazio ou por _
        para depois entrar nos links (urls) dos filmes no rotten
        """
        title = title.translate(str.maketrans({':': '', "'": '', ".":"", ",":"", 
                                            "-":"_", " ":"_"}))
        
        return title.lower()


    def create_df(self):
        links = []
        for tit in self.dataset['original_title'].apply(self.rottenize):
            links.append('https://www.rottentomatoes.com/m/{}'.format(tit))
        
        # listas para armazenar a informacao
        tmdb_id = []

        nomes = []
        TOMATOMETER = []
        TOMATOMETER_rev = []
        aud_scr = []
        aud_scr_rev = []
        main6 = []
        main7 = []
        main8 = []
        main9 = []

        genre = []
        language = []
        director = []
        producer = []
        writer = []
        rel_date_th = []
        rel_date_stream = []
        box = []
        runtime = []
        distributor = []
        description = []
        
        # neste dataset iremos armazenar as informacoes extraidas do Rotten Tomatoes
        df = pd.DataFrame()

        for j in range(len(links)):
            
            # mostra quantos filmes ja varreu
            print(j, end="\r", flush=True)
            
            # entramos no site do filme
            self.driver.get(links[j])
            
            # se aparecer algun erro no navegador voltamos ao comeco do loop
            try:
                # informacoes da descricao do filme
                info_we = self.driver.find_elements(By.CLASS_NAME, "media-body")
                # informações mais breves da descricao
                short_info = info_we[0].text.split('\n')[2:]

                # informacoes principais do filme: nome, scores e quantidade de reviews
                info_main = self.driver.find_elements(By.CLASS_NAME, "thumbnail-scoreboard-wrap")
                info_main = info_main[0].text.split('\n')

                # dicionario para armazenar as infos do filme
                short_info_dic = dict()

                # acrescentamos tambem as informacoes principais
                short_info_dic['Nome'] = info_main[0]
                short_info_dic['TOMATOMETER'] = info_main[2]
                short_info_dic['TOMATOMETER reviews'] = info_main[4]
                short_info_dic['AUDIENCE SCORE'] = info_main[5]
                short_info_dic['AUDIENCE SCORE reviews'] = info_main[7]
                
                # colunas auxiliares
                short_info_dic['main6'] = info_main[1]
                short_info_dic['main7'] = info_main[3]
                short_info_dic['main8'] = info_main[6]
                
                try:
                    short_info_dic['main9'] = info_main[8]
                except:
                    continue
                
            except:
                continue

            # informacoes breves do filme
            for i in range(len(short_info)):
                infos = short_info[i].split(': ')
                try:
                    short_info_dic[infos[0]] = infos[1]
                except:
                    short_info_dic[infos[0]] = np.nan

            # resumo e descricao do filme
            short_info_dic[info_we[0].text.split('\n')[0]] = info_we[0].text.split('\n')[1]

            # colocamos as informacoes nas listas
            try:
                nomes.append(short_info_dic['Nome'])
                tmdb_id.append(self.dataset.loc[[j],['id']].values[0][0])
            except:
                nomes.append(np.nan)
            
            try:
                TOMATOMETER.append(short_info_dic['TOMATOMETER'])
            except:
                TOMATOMETER.append(np.nan)
            
            try:
                TOMATOMETER_rev.append(short_info_dic['TOMATOMETER reviews'])
            except:
                TOMATOMETER_rev.append(np.nan)
            
            try:
                aud_scr.append(short_info_dic['AUDIENCE SCORE'])
            except:
                aud_scr.append(np.nan)
            
            try:
                main6.append(short_info_dic['main6'])
            except:
                main6.append(np.nan)
                
            try:
                main7.append(short_info_dic['main7'])
            except:
                main7.append(np.nan)
                
            try:
                main8.append(short_info_dic['main8'])
            except:
                main8.append(np.nan)
                
            try:
                main9.append(short_info_dic['main9'])
            except:
                main9.append(np.nan)
            
            try:
                aud_scr_rev.append(short_info_dic['AUDIENCE SCORE reviews'])
            except:
                aud_scr_rev.append(np.nan)
            
            try:
                genre.append(short_info_dic['Genre'])
            except:
                genre.append(np.nan)
            
            try:
                language.append(short_info_dic['Original Language'])
            except:
                language.append(np.nan)
            
            try:
                director.append(short_info_dic['Director'])
            except:
                director.append(np.nan)
            
            try:
                producer.append(short_info_dic['Producer'])
            except:
                producer.append(np.nan)
            
            try:
                writer.append(short_info_dic['Writer'])
            except:
                writer.append(np.nan)
            
            try:
                rel_date_th.append(short_info_dic['Release Date (Theaters)'])
            except:
                rel_date_th.append(np.nan)
            
            try:
                rel_date_stream.append(short_info_dic['Release Date (Streaming)'])
            except:
                rel_date_stream.append(np.nan)
            
            try:
                box.append(short_info_dic['Box Office (Gross USA)'])
            except:
                box.append(np.nan)
            
            try:
                runtime.append(short_info_dic['Runtime'])
            except:
                runtime.append(np.nan)
            
            try:
                distributor.append(short_info_dic['Distributor'])
            except:
                distributor.append(np.nan)
                
            try:
                description.append(short_info_dic['MOVIE INFO'])
            except:
                description.append(np.nan)
                
        # Alimentamos o database

        df = pd.DataFrame() 

        df['tmdb_id'] = tmdb_id
    #     df['Movie Name'] = nomes
        df['tomatometer_reviews'] = aud_scr
    #     df['AUDIENCE SCORE reviews'] = aud_scr_rev
        # mudou a estrutura do site, entao algumas listas das q foram criadas inicialmente
        # nao coincidem com o nome da coluna
        df['genre_year'] = TOMATOMETER
    #     df['TOMATOMETER reviews'] = nomes

        df['movie_name'] = main6
        df['tomatometer_score'] = main7
        df['audience_score'] = main8
        df['audience_reviews'] = main9
        
        df['Genre'] = genre
        df['Original Language'] = language
        df['Director'] = director
        df['Producer'] = producer
        df['Writer'] = writer
        df['Release Date (Theaters)'] = rel_date_th
        df['Release Date (Streaming)'] = rel_date_stream
        df['box office'] = box
        df['Runtime'] = runtime
        df['Distributor'] = distributor
        df['MOVIE INFO'] = description
            
        return df

In [4]:
dataset = pd.read_csv('ztrending_movies.csv')

In [ ]:
dataset

In [5]:
rotten = RottenCrawler(dataset, True)

[WDM] - Downloading:  96%|█████████▌| 9.13M/9.52M [00:01<00:00, 6.57MB/s]<ipython-input-2-ed605a50f0d0>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=driver_options)
<ipython-input-2-ed605a50f0d0>:8: DeprecationWarning: use options instead of chrome_options
  self.driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=driver_options)


In [6]:
df = rotten.create_df()

[WDM] - Downloading: 100%|██████████| 9.52M/9.52M [00:19<00:00, 6.57MB/s]

In [7]:
df

,tmdb_id,tomatometer_reviews,genre_year,movie_name,tomatometer_score,audience_score,audience_reviews,Genre,Original Language,Director,Producer,Writer,Release Date (Theaters),Release Date (Streaming),box office,Runtime,Distributor,MOVIE INFO
0,299537,550 Reviews,"PG-13 2019, Action/Adventure, 2h 4m",CAPTAIN MARVEL,79%,45%,"100,000+ Ratings","Action, Adventure, Sci-Fi, Fantasy",English,"Anna Boden, Ryan Fleck",Kevin Feige,"Anna Boden, Ryan Fleck, Geneva Robertson-Dwore...","Mar 8, 2019 Wide","May 28, 2019",$426.8M,2h 4m,Walt Disney,Captain Marvel is an extraterrestrial Kree war...
1,335984,444 Reviews,"R 2017, Sci-fi/Mystery & thriller, 2h 44m",BLADE RUNNER 2049,88%,82%,"50,000+ Ratings","Sci-Fi, Mystery & Thriller",English,Denis Villeneuve,"Andrew A. Kosove, Broderick Johnson, Bud Yorki...","Hampton Fancher, Michael Green","Oct 6, 2017 Wide","Jan 16, 2018",$91.5M,2h 44m,Warner Bros. Pictures,"Officer K (Ryan Gosling), a new blade runner f..."
2,1771,274 Reviews,"PG-13 2011, Action/Adventure, 2h 4m",CAPTAIN AMERICA: THE FIRST AVENGER,80%,75%,"100,000+ Ratings","Action, Adventure, Fantasy",English,Joe Johnston,Kevin Feige,"Christopher Markus, Stephen McFeely","Jul 22, 2011 Wide","Oct 25, 2011",$176.6M,2h 4m,Paramount Pictures,It is 1941 and the world is in the throes of w...
3,2034,165 Reviews,"R 2001, Crime/Drama, 2h 0m",TRAINING DAY,74%,89%,"250,000+ Ratings","Crime, Drama, Mystery & Thriller",English,Antoine Fuqua,"Jeffrey Silver, Bobby Newmyer",David Ayer,"Oct 5, 2001 Wide","Mar 19, 2002",$76.3M,2h 0m,Warner Bros. Pictures,Police drama about a veteran officer who escor...
4,121,256 Reviews,"PG-13 2002, Fantasy/Adventure, 2h 59m",THE LORD OF THE RINGS: THE TWO TOWERS,95%,95%,"250,000+ Ratings","Fantasy, Adventure",English,Peter Jackson,"Peter Jackson, Barrie M. Osborne, Fran Walsh, ...","Peter Jackson, Philippa Boyens, Fran Walsh, St...","Dec 18, 2002 Wide","Aug 26, 2003",$342.6M,2h 59m,New Line Cinema,The sequel to the Golden Globe-nominated and A...
5,872954,56 Reviews,"R 2023, Western/Action, 1h 35m",THE OLD WAY,32%,70%,Fewer than 50 Verified Ratings,"Western, Action",English,Brett Donowho,"Brett Donowho, Micah Haley, Robert Paschall Jr...",Carl W. Lucas,"Jan 6, 2023 Limited","Jan 13, 2023",$365.0K,1h 35m,Saban Films,Academy Award® winner Nicolas Cage stars in hi...
6,1061163,35 Reviews,"R 2023, Romance/Comedy, 1h 40m",AT MIDNIGHT,43%,86%,100+ Ratings,"Romance, Comedy",English,Jonah Feingold,"Fred Berger, David Bernon, Brian Kavanaugh-Jon...","Jonah Feingold, María Hinojos, Giovanni M. Porta",NaN,"Feb 10, 2023",NaN,1h 40m,NaN,Alejandro (Diego Boneta) is a hotel manager. S...
